In [1]:
import pandas as pd
direccion = "./../src/data/raw/prueba.csv"
df = pd.read_csv(direccion, nrows = 10000)
df.head()

df.Cancelled

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Name: Cancelled, Length: 10000, dtype: float64

In [2]:
direccion = "./../src/data/raw/prueba.csv"

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.csv(direccion, header="true", inferSchema="true").limit(10000)

In [3]:
def clean(df):
    #Pasar a minusculas los nombres de columnas
    for col in df.columns:
        df = df.withColumnRenamed(col, col.lower())

    #Seleccionar columnas no vacias 

    base = df.select(df.year,df.quarter, df.month, df.dayofmonth, df.dayofweek, df.flightdate, df.reporting_airline, df.dot_id_reporting_airline, df.iata_code_reporting_airline, df.tail_number, df.flight_number_reporting_airline, df.originairportid, df.originairportseqid, df.origincitymarketid, df.origin, df.origincityname, df.originstate, df.originstatefips, df.originstatename, df.originwac, df.destairportid, df.destairportseqid, df.destcitymarketid, df.dest, df.destcityname, df.deststate, df.deststatefips, df.deststatename, df.destwac, df.crsdeptime, df.deptime, df.depdelay, df.depdelayminutes, df.depdel15, df.departuredelaygroups, df.deptimeblk, df.taxiout, df.wheelsoff, df.wheelson, df.taxiin, df.crsarrtime, df.arrtime, df.arrdelay, df.arrdelayminutes, df.arrdel15, df.arrivaldelaygroups, df.arrtimeblk, df.cancelled, df.diverted, df.crselapsedtime, df.actualelapsedtime, df.airtime, df.flights, df.distance, df.distancegroup, df.divairportlandings )

    #agregar columna con clasificación de tiempo en horas de atraso del vuelo 0-1.5, 1.5-3.5,3.5-, cancelled

    from pyspark.sql import functions as f
    base = base.withColumn('rangoatrasohoras', f.when(f.col('cancelled') == 1, "cancelled").when(f.col('depdelayminutes') < 90, "0-1.5").when((f.col('depdelayminutes') > 90) & (f.col('depdelayminutes')<210), "1.5-3.5").otherwise("3.5-"))

    from pyspark.sql.functions import udf
    from pyspark.sql.types import StringType
    from pyspark.sql.functions import col, lower, regexp_replace, split

    #Función limpieza
    def clean_text(c):
        c = lower(c)
        c = regexp_replace(c, " ", "_")
        c = f.split(c, '\,')[0]
        return c


     # Aplicación de la función limpieza
    base = base.withColumn("origincityname", clean_text(col("origincityname")))
    base = base.withColumn("destcityname", clean_text(col("destcityname")))
    return base

In [4]:
df_clean = clean(df)

In [13]:
df_num.columns

['year',
 'quarter',
 'month',
 'dayofmonth',
 'dayofweek',
 'dot_id_reporting_airline',
 'flight_number_reporting_airline',
 'originairportid',
 'originairportseqid',
 'origincitymarketid',
 'originstatefips',
 'originwac',
 'destairportid',
 'destairportseqid',
 'destcitymarketid',
 'deststatefips',
 'destwac',
 'crsdeptime',
 'deptime',
 'departuredelaygroups',
 'wheelsoff',
 'wheelson',
 'crsarrtime',
 'arrtime',
 'arrivaldelaygroups',
 'distancegroup',
 'divairportlandings',
 'depdelay',
 'depdelayminutes',
 'depdel15',
 'taxiout',
 'taxiin',
 'arrdelay',
 'arrdelayminutes',
 'arrdel15',
 'cancelled',
 'diverted',
 'crselapsedtime',
 'actualelapsedtime',
 'airtime',
 'flights',
 'distance',
 'rangoatrasohoras']

In [5]:
type(df_clean)
from pyspark.sql.types import IntegerType, DoubleType

double_cols = [f.name for f in df_clean.schema.fields if isinstance(f.dataType, DoubleType)]
int_cols = [f.name for f in df_clean.schema.fields if isinstance(f.dataType, IntegerType)]
num_cols = int_cols + double_cols

cols = num_cols + ['rangoatrasohoras']

df_num = df_clean[cols]

# MISSING VALUES
# POR AHORA ZEROS (MANEJAR ESTO DESPUES)
#df_na = df_num.fill(0)

numericals_imputed = [var + "_imputed" for var in double_cols]

from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = double_cols, outputCols = numericals_imputed)
df_impute = imputer.setStrategy("median").fit(df_num).transform(df_num)
new_vars = numericals_imputed + int_cols + ['rangoatrasohoras']
df_impute = df_impute[new_vars]

df_impute = df_impute.fillna(0, subset=int_cols)

df_impute.columns


['depdelay_imputed',
 'depdelayminutes_imputed',
 'depdel15_imputed',
 'taxiout_imputed',
 'taxiin_imputed',
 'arrdelay_imputed',
 'arrdelayminutes_imputed',
 'arrdel15_imputed',
 'cancelled_imputed',
 'diverted_imputed',
 'crselapsedtime_imputed',
 'actualelapsedtime_imputed',
 'airtime_imputed',
 'flights_imputed',
 'distance_imputed',
 'year',
 'quarter',
 'month',
 'dayofmonth',
 'dayofweek',
 'dot_id_reporting_airline',
 'flight_number_reporting_airline',
 'originairportid',
 'originairportseqid',
 'origincitymarketid',
 'originstatefips',
 'originwac',
 'destairportid',
 'destairportseqid',
 'destcitymarketid',
 'deststatefips',
 'destwac',
 'crsdeptime',
 'deptime',
 'departuredelaygroups',
 'wheelsoff',
 'wheelson',
 'crsarrtime',
 'arrtime',
 'arrivaldelaygroups',
 'distancegroup',
 'divairportlandings',
 'rangoatrasohoras']

In [6]:
from pyspark.sql.types import LongType
from pyspark.sql import Row
def zipindexdf(df):
    schema_new = df.schema.add("index", LongType(), False)
    return df.rdd.zipWithIndex().map(lambda l: list(l[0]) + [l[1]]).toDF(schema_new)

unas = df_impute.select(df_impute.year,df_impute.quarter, df_impute.month, df_impute.dayofmonth, df_impute.dayofweek, df_impute.crsdeptime, df_impute.dot_id_reporting_airline, df_impute.origincitymarketid, df_impute.originairportseqid, df_impute.originwac, df_impute.destcitymarketid, df_impute.destairportseqid, df_impute.destwac, df_impute.flights_imputed, df_impute.rangoatrasohoras, df_impute.depdelayminutes_imputed, df_impute.arrdelayminutes_imputed, df_impute.actualelapsedtime_imputed, df_impute.airtime_imputed, df_impute.distance_imputed)
otras = df_clean.select(df_clean.tail_number,df_clean.deststate, df_clean.originstate)

df1_index = zipindexdf(unas)

df2_index = zipindexdf(otras)

df_id = df1_index.join(df2_index, "index", "inner")

In [87]:
# Sum by Agg
import pyspark.sql.functions as func
from pyspark.sql.functions import desc, col
group_data = df_id.groupBy("originwac")
orwac_delay = group_data.agg(func.sum("depdelayminutes_imputed").alias("sum_depdelayminutes_imputed")).sort(desc("sum_depdelayminutes_imputed"))
orwac_delay.show(5)
#wac_delay.orderBy(wac_delay['sum_depdelayminutes_imputed']).desc().show()

+---------+---------------------------+
|originwac|sum_depdelayminutes_imputed|
+---------+---------------------------+
|       91|                    16234.0|
|       41|                    11008.0|
|       43|                     6045.0|
|       82|                     4811.0|
|       74|                     4685.0|
+---------+---------------------------+
only showing top 5 rows



In [100]:
group_data = df_id.groupBy("destwac")
destwac_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
destwac_delay.show(5)

+-------+---------------------------+
|destwac|sum_arrdelayminutes_imputed|
+-------+---------------------------+
|     91|                    14154.0|
|     41|                    11422.0|
|     43|                     8824.0|
|     82|                     5780.0|
|     74|                     4461.0|
+-------+---------------------------+
only showing top 5 rows



In [95]:
group_data = df_id.groupBy("originairportseqid")
orcity_delay = group_data.agg(func.sum("depdelayminutes_imputed").alias("sum_depdelayminutes_imputed")).sort(desc("sum_depdelayminutes_imputed"))
orcity_delay.show(5)

+------------------+---------------------------+
|originairportseqid|sum_depdelayminutes_imputed|
+------------------+---------------------------+
|           1393007|                    10395.0|
|           1477104|                     6996.0|
|           1129202|                     3691.0|
|           1143302|                     3414.0|
|           1289208|                     2603.0|
+------------------+---------------------------+
only showing top 5 rows



In [99]:
group_data = df_id.groupBy("destairportseqid")
descity_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
descity_delay.show(5)

+----------------+---------------------------+
|destairportseqid|sum_arrdelayminutes_imputed|
+----------------+---------------------------+
|         1393007|                    11055.0|
|         1477104|                     4906.0|
|         1143302|                     4629.0|
|         1129202|                     4358.0|
|         1348702|                     2860.0|
+----------------+---------------------------+
only showing top 5 rows



In [96]:
group_data = df_id.groupBy("originairportseqid")
orairp_delay = group_data.agg(func.sum("depdelayminutes_imputed").alias("sum_depdelayminutes_imputed")).sort(desc("sum_depdelayminutes_imputed"))
orairp_delay.show(5)

+------------------+---------------------------+
|originairportseqid|sum_depdelayminutes_imputed|
+------------------+---------------------------+
|           1393007|                    10395.0|
|           1477104|                     6996.0|
|           1129202|                     3691.0|
|           1143302|                     3414.0|
|           1289208|                     2603.0|
+------------------+---------------------------+
only showing top 5 rows



In [101]:
group_data = df_id.groupBy("destairportseqid")
desairp_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
desairp_delay.show(5)

+----------------+---------------------------+
|destairportseqid|sum_arrdelayminutes_imputed|
+----------------+---------------------------+
|         1393007|                    11055.0|
|         1477104|                     4906.0|
|         1143302|                     4629.0|
|         1129202|                     4358.0|
|         1348702|                     2860.0|
+----------------+---------------------------+
only showing top 5 rows



In [104]:
group_data = df_id.groupBy("dayofweek")
orairp_delay = group_data.agg(func.sum("depdelayminutes_imputed").alias("sum_depdelayminutes_imputed")).sort(desc("sum_depdelayminutes_imputed"))
orairp_delay.show(5)

+---------+---------------------------+
|dayofweek|sum_depdelayminutes_imputed|
+---------+---------------------------+
|        5|                    26093.0|
|        7|                    19108.0|
|        6|                    16547.0|
|        4|                    15012.0|
|        1|                     9740.0|
+---------+---------------------------+
only showing top 5 rows



In [105]:
group_data = df_id.groupBy("dayofweek")
desairp_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
desairp_delay.show(5)

+---------+---------------------------+
|dayofweek|sum_arrdelayminutes_imputed|
+---------+---------------------------+
|        5|                    24754.0|
|        7|                    17544.0|
|        6|                    15328.0|
|        4|                    14332.0|
|        1|                     8821.0|
+---------+---------------------------+
only showing top 5 rows



In [106]:
group_data = df_id.groupBy("dayofmonth")
orairp_delay = group_data.agg(func.sum("depdelayminutes_imputed").alias("sum_depdelayminutes_imputed")).sort(desc("sum_depdelayminutes_imputed"))
orairp_delay.show(5)

+----------+---------------------------+
|dayofmonth|sum_depdelayminutes_imputed|
+----------+---------------------------+
|         1|                    21497.0|
|         3|                    18636.0|
|         2|                    15333.0|
|        14|                    14644.0|
|         4|                     8719.0|
+----------+---------------------------+
only showing top 5 rows



In [110]:
group_data = df_id.groupBy("dayofmonth")
desairp_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
desairp_delay.show(5)

+----------+---------------------------+
|dayofmonth|sum_arrdelayminutes_imputed|
+----------+---------------------------+
|         1|                    19799.0|
|         3|                    17151.0|
|         2|                    14130.0|
|        14|                    13990.0|
|         4|                     7828.0|
+----------+---------------------------+
only showing top 5 rows



In [111]:
group_data = df_id.groupBy("originstate")
desairp_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
desairp_delay.show(5)

+-----------+---------------------------+
|originstate|sum_arrdelayminutes_imputed|
+-----------+---------------------------+
|         CA|                    15013.0|
|         IL|                    10257.0|
|         MI|                     5068.0|
|         TX|                     4774.0|
|         CO|                     4096.0|
+-----------+---------------------------+
only showing top 5 rows



In [112]:
group_data = df_id.groupBy("deststate")
desairp_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
desairp_delay.show(5)

+---------+---------------------------+
|deststate|sum_arrdelayminutes_imputed|
+---------+---------------------------+
|       CA|                    14154.0|
|       IL|                    11422.0|
|       MI|                     8824.0|
|       CO|                     5780.0|
|       TX|                     4461.0|
+---------+---------------------------+
only showing top 5 rows



In [115]:
group_data = df_id.groupBy("dot_id_reporting_airline")
desairp_delay = group_data.agg(func.sum("arrdelayminutes_imputed").alias("sum_arrdelayminutes_imputed")).sort(desc("sum_arrdelayminutes_imputed"))
desairp_delay.show(5)

+------------------------+---------------------------+
|dot_id_reporting_airline|sum_arrdelayminutes_imputed|
+------------------------+---------------------------+
|                   20304|                    76681.0|
|                   20397|                     6381.0|
+------------------------+---------------------------+



In [163]:
df_id = df_id.withColumn('dephour',f.when(f.length('crsdeptime')==3,df_id['crsdeptime'].substr(0,1)).otherwise(df_id['crsdeptime'].substr(0,2)) )
df_id.select('dephour').distinct().count()

19

In [167]:
from pyspark.sql import functions as f
group_data = df_id.groupBy("dephour")
orairp_delay = group_data.agg(f.sum("depdelayminutes_imputed").alias("sum_depdelayminutes_imputed")).sort(desc("sum_depdelayminutes_imputed"))
orairp_delay.show(10)

+-------+---------------------------+
|dephour|sum_depdelayminutes_imputed|
+-------+---------------------------+
|     18|                     8263.0|
|      6|                     6736.0|
|     12|                     6541.0|
|      8|                     6391.0|
|     10|                     5813.0|
|     11|                     5733.0|
|     15|                     5517.0|
|     16|                     5248.0|
|     14|                     5168.0|
|     17|                     5125.0|
+-------+---------------------------+
only showing top 10 rows



In [66]:
df_id.collect()[0:1][:]
print(('Filas:',df_id.count(),'Columnas:',len(df_id.columns)))
df_id.columns

('Filas:', 10000, 'Columnas:', 24)


['index',
 'year',
 'quarter',
 'month',
 'dayofmonth',
 'dayofweek',
 'crsdeptime',
 'dot_id_reporting_airline',
 'origincitymarketid',
 'originairportseqid',
 'originwac',
 'destcitymarketid',
 'destairportseqid',
 'destwac',
 'flights_imputed',
 'rangoatrasohoras',
 'depdelayminutes_imputed',
 'arrdelayminutes_imputed',
 'actualelapsedtime_imputed',
 'airtime_imputed',
 'distance_imputed',
 'tail_number',
 'deststate',
 'originstate']

In [8]:
def crear_features(base):
    from pyspark.sql import functions as f
    base = base.withColumn('findesemana', f.when(f.col('dayofweek') == 5, 1).when(f.col('dayofweek') == 6, 1).when(f.col('dayofweek') == 7, 1).otherwise(0))
    base = base.withColumn('quincena', f.when(f.col('dayofmonth') == 15, 1).when(f.col('dayofmonth') == 14, 1).when(f.col('dayofmonth') == 16, 1).when(f.col('dayofmonth') == 29, 1).when(f.col('dayofmonth') == 30, 1).when(f.col('dayofmonth') == 31, 1).when(f.col('dayofmonth') == 1, 1).when(f.col('dayofmonth') == 2, 1).when(f.col('dayofmonth') == 3, 1).otherwise(0))
    base = base.withColumn('dephour',f.when(f.length('crsdeptime')==3,f.col('crsdeptime').substr(0,1).cast("float")).otherwise(f.col('crsdeptime').substr(0,2).cast("float")) )
    base = base.withColumn('seishoras', f.when(f.col('dephour') == 6, 1).when(f.col('dephour') == 12, 1).when(f.col('dephour') == 18, 1).when(f.col('dephour') == 0, 1).otherwise(0))
    return(base)

In [14]:
df_util = crear_features(df_num)
df_util = df_util.drop('tail_number')
df_util = df_util.drop('originstate')
df_util = df_util.drop('deststate')
df_util = df_util.drop('dayofweek')
df_util = df_util.drop('dayofmonth')
df_util = df_util.drop('dephour')

In [15]:
df_util.head()

Row(year=2019, quarter=4, month=11, dot_id_reporting_airline=20397, flight_number_reporting_airline=5213, originairportid=13795, originairportseqid=1379502, origincitymarketid=33795, originstatefips=37, originwac=36, destairportid=11057, destairportseqid=1105703, destcitymarketid=31057, deststatefips=37, destwac=36, crsdeptime=1036, deptime=1032, departuredelaygroups=-1, wheelsoff=1041, wheelson=1119, crsarrtime=1201, arrtime=1127, arrivaldelaygroups=-2, distancegroup=1, divairportlandings=0, depdelay=-4.0, depdelayminutes=0.0, depdel15=0.0, taxiout=9.0, taxiin=8.0, arrdelay=-34.0, arrdelayminutes=0.0, arrdel15=0.0, cancelled=0.0, diverted=0.0, crselapsedtime=85.0, actualelapsedtime=55.0, airtime=38.0, flights=1.0, distance=191.0, rangoatrasohoras='0-1.5', findesemana=1, quincena=1, seishoras=0)

In [206]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
df_util  = df_util.withColumn("index",monotonically_increasing_id())

nuevas_vars =  [x for x in df_util.columns if x not in ['rangoatrasohoras']]

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=nuevas_vars ,outputCol="features")
assembler = assembler.transform(df_util)

keep_vars = ["index", "features", 'rangoatrasohoras']
df_assembler = assembler[keep_vars]
assembler.show(2)

+----------------+-----------------------+----------------+---------------+--------------+----------------+-----------------------+----------------+-----------------+----------------+----------------------+-------------------------+---------------+---------------+----------------+----+-------+-----+------------------------+-------------------------------+---------------+------------------+------------------+---------------+---------+-------------+----------------+----------------+-------------+-------+----------+-------+--------------------+---------+--------+----------+-------+------------------+-------------+------------------+----------------+-----------+--------+---------+-----+--------------------+
|depdelay_imputed|depdelayminutes_imputed|depdel15_imputed|taxiout_imputed|taxiin_imputed|arrdelay_imputed|arrdelayminutes_imputed|arrdel15_imputed|cancelled_imputed|diverted_imputed|crselapsedtime_imputed|actualelapsedtime_imputed|airtime_imputed|flights_imputed|distance_imputed|year|q

## Modelado

###  Formato al Dataframe 

In [207]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer


# Esta funcion agrega un id numerico a cada valor diferente de un valor categorico 
# es como establecer los niveles en R de una factor pero los niveles son numericos,
# sus id. El indice se establece por orden de frecuencia (descendente), por lo que 
# el indice 0 corresponde a la variable que aparece con mas frecuencia
string_indexer = StringIndexer(inputCol='rangoatrasohoras', outputCol="categoryIndex")
model = string_indexer.fit(df_assembler)
indexed = model.transform(df_assembler)

# Antes falta escalar variables
# OneHotEncoder no tiene un fit ya que solo es un transformador
encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed )
df_encoded = encoded[["index", "features", "categoryIndex","categoryVec"]]

df_encoded.show(10)

+-----+--------------------+-------------+-------------+
|index|            features|categoryIndex|  categoryVec|
+-----+--------------------+-------------+-------------+
|    0|[-4.0,0.0,0.0,9.0...|          0.0|(3,[0],[1.0])|
|    1|[18.0,18.0,1.0,13...|          0.0|(3,[0],[1.0])|
|    2|[36.0,36.0,1.0,12...|          0.0|(3,[0],[1.0])|
|    3|[36.0,36.0,1.0,15...|          0.0|(3,[0],[1.0])|
|    4|[-7.0,0.0,0.0,8.0...|          0.0|(3,[0],[1.0])|
|    5|[-4.0,0.0,0.0,22....|          0.0|(3,[0],[1.0])|
|    6|[1.0,1.0,0.0,11.0...|          0.0|(3,[0],[1.0])|
|    7|[12.0,12.0,0.0,17...|          0.0|(3,[0],[1.0])|
|    8|[166.0,166.0,1.0,...|          1.0|(3,[1],[1.0])|
|    9|[-2.0,0.0,0.0,22....|          0.0|(3,[0],[1.0])|
+-----+--------------------+-------------+-------------+
only showing top 10 rows



In [208]:
# Podemos empezar con el modelado
#from pyspark.sql import randomSplit

# Separamos en train y test
df_train, df_test = df_encoded.randomSplit([0.8,0.2], 123)
df_test.show(2)

+-----+--------------------+-------------+-------------+
|index|            features|categoryIndex|  categoryVec|
+-----+--------------------+-------------+-------------+
|    1|[18.0,18.0,1.0,13...|          0.0|(3,[0],[1.0])|
|    2|[36.0,36.0,1.0,12...|          0.0|(3,[0],[1.0])|
+-----+--------------------+-------------+-------------+
only showing top 2 rows



In [209]:

from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit


selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="categoryIndex")

selector_model = selector.fit(df_train)

selector_result = selector_model.transform(df_train)
print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())

df_train = selector_result[["index", "selectedFeatures", "categoryIndex","categoryVec"]]

df_train.show(2)

selector_result2 = selector_model.transform(df_test)

df_test = selector_result2[["index", "selectedFeatures", "categoryIndex","categoryVec"]]


ChiSqSelector output with top 10 features selected
+-----+--------------------+-------------+-------------+
|index|    selectedFeatures|categoryIndex|  categoryVec|
+-----+--------------------+-------------+-------------+
|    0|[-4.0,0.0,0.0,9.0...|          0.0|(3,[0],[1.0])|
|    4|[-7.0,0.0,0.0,8.0...|          0.0|(3,[0],[1.0])|
+-----+--------------------+-------------+-------------+
only showing top 2 rows



In [210]:
# Configuramos el estimator StandarScaler como lo necesitamos (por default
# withMean esta en False porque hace que se regrese un vector dense...
# hay que tener cuidado con eso cuando estemos manejandoo vectores sparse
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="selectedFeatures", outputCol="scaled_features",
                        withStd=True, withMean=True)
# Creamos el modelo StandardScaler para los datos de entrada
scaler_model = scaler.fit(df_train)

# Transformamos los datos 
scaled_data = scaler_model.transform(df_train)

df_train = scaled_data[["index", "scaled_features", "categoryIndex","categoryVec"]]
df_train.show(2)

scaled_data_test = scaler_model.transform(df_test)
df_test = scaled_data_test[["index", "scaled_features", "categoryIndex","categoryVec"]]


+-----+--------------------+-------------+-------------+
|index|     scaled_features|categoryIndex|  categoryVec|
+-----+--------------------+-------------+-------------+
|    0|[-0.1967629108606...|          0.0|(3,[0],[1.0])|
|    4|[-0.2663808227574...|          0.0|(3,[0],[1.0])|
+-----+--------------------+-------------+-------------+
only showing top 2 rows



In [211]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, regParam=0.01, fitIntercept=True , 
                        labelCol ="categoryIndex", featuresCol = "scaled_features"  )

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr,  labelCol ="categoryIndex", featuresCol = "scaled_features" )

# train the multiclass model.
ovrModel = ovr.fit(df_train)

# score the model on test data.
predictions = ovrModel.transform(df_test)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy",labelCol ="categoryIndex")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions )
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0126459


## Util para 

In [212]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegression, OneVsRest

lr = LogisticRegression(maxIter=10, regParam=0.01,
                       labelCol ="categoryVec", featuresCol = "scaled_features"  )

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")
print("LogisticRegression parameters:\n" +evaluator.explainParams() + "\n")


LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: scaled_features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: categoryVec)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitti